In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install textblob

In [ ]:
import pandas as pd
import json
import re

# Games

In [ ]:
# Cargar '/content/drive/MyDrive/pf_henry/datasets/output_steam_games.json' en un DataFrame
data = []
with open('/content/drive/MyDrive/pf_henry/datasets/output_steam_games.json') as f:
    for line in f:
        data.append(json.loads(line))

df_games = pd.DataFrame(data)

In [ ]:
df_games_clean = df_games[df_games['genres'].notna()]

In [ ]:
# Comprobar si existen registros donde 'title' o 'id' son 'NaN'
if not df_games_clean.empty:
    num_nan_title = df_games_clean['title'].isna().sum()
    num_nan_id = df_games_clean['id'].isna().sum()
    print(f"Existen registros con 'NaN' en 'title'. Cantidad: {num_nan_title}")
    print(f"Existen registros con 'NaN' en 'id'. Cantidad: {num_nan_id}")
else:
    print("No existen registros con 'NaN' en 'title' o 'id'.")

Existen registros con 'NaN' en 'title'. Cantidad: 1
Existen registros con 'NaN' en 'id'. Cantidad: 1


In [ ]:
print("\nRegistros con 'NaN' en 'id' o en 'title':")
for index, row in df_games_clean.iterrows():
  if pd.isna(row['id']) | pd.isna(row['title']):
    print(row)


Registros con 'NaN' en 'id' o en 'title':


In [ ]:
df_games_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28852 entries, 88310 to 120443
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publisher       23955 non-null  object 
 1   genres          28852 non-null  object 
 2   app_name        28851 non-null  object 
 3   title           28852 non-null  object 
 4   url             28852 non-null  object 
 5   release_date    28834 non-null  object 
 6   tags            28828 non-null  object 
 7   reviews_url     28851 non-null  object 
 8   discount_price  204 non-null    float64
 9   specs           28566 non-null  object 
 10  price           27622 non-null  object 
 11  early_access    28852 non-null  object 
 12  id              28852 non-null  object 
 13  metascore       2613 non-null   object 
 14  developer       28682 non-null  object 
 15  user_id         0 non-null      object 
 16  steam_id        0 non-null      object 
 17  items           0 non-null

In [ ]:
# Crear una copia independiente del DataFrame
df_games_clean = df_games_clean.copy()

# Función para buscar y cambiar fechas en formato 'month yyyy' a 'yyyy-mm-01'
def convert_month_year_to_yyyy_mm_01(date_str):
    try:
        # Intentar encontrar el patrón 'month yyyy' en el texto
        match = re.search(r'(\w+)\s+(\d{4})', date_str)

        if match:
            # Obtener el mes y el año coincidentes
            month, year = match.groups()

            # Mapear nombres de meses a números
            month_dict = {
                'January': '01', 'February': '02', 'March': '03', 'April': '04',
                'May': '05', 'June': '06', 'July': '07', 'August': '08',
                'September': '09', 'October': '10', 'November': '11', 'December': '12'
            }

            # Componer la fecha en formato 'yyyy-mm-01'
            yyyy_mm_01 = f"{year}-{month_dict[month]}-01"
            return yyyy_mm_01
    except:
        pass

    # Si no se encontró un formato válido, mantener el valor original
    return date_str

# Buscar y cambiar registros en la columna 'release_date'
df_games_clean['release_date'] = df_games_clean['release_date'].apply(convert_month_year_to_yyyy_mm_01)

In [ ]:
# Función para buscar y cambiar fechas en formato 'yyyy' a 'yyyy-01-01'
def convert_year_to_yyyy_01_01(date_str):
    try:
        # Intentar encontrar el patrón '\d{4}' (año de 4 dígitos) en el texto
        match = re.search(r'(\d{4})', date_str)

        if match:
            # Obtener el año coincidente
            year = match.group(1)

            # Componer la fecha en formato 'yyyy-01-01'
            yyyy_01_01 = f"{year}-01-01"
            return yyyy_01_01
    except:
        pass

    # Si no se encontró un formato válido, mantener el valor original
    return date_str

# Buscar y cambiar registros en la columna 'release_date'
df_games_clean['release_date'] = df_games_clean['release_date'].apply(convert_year_to_yyyy_01_01)

In [ ]:
# Función para reemplazar valores que contienen 'soon' (mayúsculas o minúsculas)
def replace_soon_with_0000_00_00(date_str):
    # Utilizar una expresión regular para buscar 'soon' (ignorar mayúsculas/minúsculas y otros caracteres)
    pattern = re.compile(r'\bsoon\b', re.IGNORECASE)
    if pd.notna(date_str) and pattern.search(date_str):
        return '0000-00-00'
    return date_str

# Buscar y cambiar registros en la columna 'release_date'
df_games_clean['release_date'] = df_games_clean['release_date'].apply(replace_soon_with_0000_00_00)

In [ ]:
# Convertir la columna 'release_date' a tipo string
df_games_clean['release_date'] = df_games_clean['release_date'].astype(str)

# Condicional para identificar registros que no cumplen con el formato 'yyyy-mm-dd'
condition = ~df_games_clean['release_date'].str.match(r'^\d{4}-\d{2}-\d{2}$')

# Cambiar el valor de 'release_date' por '0000-00-00' en los registros que cumplan la condición
df_games_clean.loc[condition, 'release_date'] = '0000-00-00'

In [ ]:
# Luego, convertimos la columna 'genres' en columnas binarias
df_genres = df_games_clean['genres'].apply(lambda x: pd.Series([1] * len(x), index=x) if isinstance(x, list) else x).fillna(0)

# Unimos las columnas binarias al DataFrame original
df_games_clean = pd.concat([df_games_clean, df_genres], axis=1)

# Identificamos las columnas nuevas creadas por df_genres
new_genre_columns = df_genres.columns

# Cambiamos el tipo de las columnas nuevas a int
df_games_clean[new_genre_columns] = df_games_clean[new_genre_columns].astype(int)

# Reemplazamos espacios en los nombres de las columnas por guiones bajos
df_games_clean.columns = df_games_clean.columns.str.replace('&amp;', '')
df_games_clean.columns = df_games_clean.columns.str.replace(' ', '_')

In [ ]:
df_games_final = df_games_clean[['id', 'title', 'genres', 'release_date']]

In [ ]:
df_games_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28852 entries, 88310 to 120443
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            28852 non-null  object
 1   title         28852 non-null  object
 2   genres        28852 non-null  object
 3   release_date  28852 non-null  object
dtypes: object(4)
memory usage: 1.1+ MB


In [ ]:
# Convertir la columna 'release_date' a cadenas de caracteres (strings)
df_games_final['release_date'] = df_games_final['release_date'].astype(str)

# Filtrar las filas donde 'release_date' no cumple con la estructura 'yyyy-mm-dd'
invalid_dates = df_games_final[~df_games_final['release_date'].str.match(r'^\d{4}-\d{2}-\d{2}$')]

count_invalid_dates = len(invalid_dates)
print(f"{count_invalid_dates} registros no cumplen con la estructura 'yyyy-mm-dd' en la columna 'release_date'.")

# Mostrar los registros que no cumplen
print("\nRegistros que no cumplen con la estructura 'yyyy-mm-dd' en la columna 'release_date':")
print(invalid_dates)

0 registros no cumplen con la estructura 'yyyy-mm-dd' en la columna 'release_date'.

Registros que no cumplen con la estructura 'yyyy-mm-dd' en la columna 'release_date':
Empty DataFrame
Columns: [id, title, genres, release_date]
Index: []


<ipython-input-37-2dd0db9eee7c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_games_final['release_date'] = df_games_final['release_date'].astype(str)


In [ ]:
# Guardar el DataFrame como un archivo CSV en la misma ruta
df_games_final.to_csv('/content/drive/MyDrive/pf_henry/datasets/games.csv', index=False)

# itmes

In [ ]:
with open('/content/drive/MyDrive/pf_henry/datasets/australian_users_items.json', 'r', encoding='utf-8') as f:
    data =f.readlines()

records = [eval(line.strip()) for line in data]

df_items = pd.DataFrame(records)

In [ ]:
df_items.shape

(88310, 5)

In [ ]:
df_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [ ]:
# Crear el DataFrame con las columnas user_id, items_count, steam_id y user_url
df_users_info = df_items[['user_id', 'items_count', 'steam_id', 'user_url']]

# Crear una lista para almacenar los registros desglosados de items
items_records = []

# Recorrer el DataFrame original y desglosar los elementos de items
for index, row in df_items.iterrows():
    user_id = row['user_id']
    items_list = row['items']
    for item in items_list:
        item_record = {
            'user_id': user_id,
            'item_id': item['item_id'],
            # 'item_name': item['item_name'],
            'playtime_forever': item['playtime_forever'],
            # 'playtime_2weeks': item['playtime_2weeks']
        }
        items_records.append(item_record)

# Crear el DataFrame con los registros desglosados de items
df_items_info = pd.DataFrame(items_records)

In [ ]:
# Filtrar las filas donde 'playtime_forever' sea mayor a 1200
df_items = df_items[df_items['playtime_forever'] > 1200]
df_items.info()

In [ ]:
df_items_info.shape

(5153209, 3)

In [ ]:
df_items_info.iloc[0]

user_id             76561197970982479
item_id                            10
playtime_forever                    6
Name: 0, dtype: object

In [ ]:
# Guardar el DataFrame como un archivo CSV en la misma ruta
df_items_info.to_csv('/content/drive/MyDrive/pf_henry/datasets/items.csv', index=False)

# Reviews

In [ ]:
# Abre el archivo JSON y lee los datos
with open('/content/drive/MyDrive/pf_henry/datasets/australian_user_reviews.json', 'r', encoding='utf-8') as f:
    data = f.readlines()

# Corrige el formato JSON eliminando comas al final de las líneas y encierra todo en corchetes para formar una lista de diccionarios válida
fixed_data = "[" + ",".join(data) + "]"

# Convierte el JSON en una lista de diccionarios
records = eval(fixed_data)

# Crea el DataFrame con las reviews y agrega la relación con user_id
reviews_df = pd.DataFrame([{
    "user_id": record["user_id"],
    "item_id": review["item_id"],
    "posted": review["posted"],
    "recommend": review["recommend"],
    "review": review["review"]
} for record in records for review in record["reviews"]])

In [ ]:
# Extraer la parte de la fecha de la columna "posted" usando una expresión regular
reviews_df['posted'] = reviews_df['posted'].str.extract(r'Posted (.+?)\.')

# Definir un formato que permita fechas con o sin año
date_formats = ['%B %d, %Y', '%B %d']

# Intentar convertir la columna "posted" al formato deseado
for date_format in date_formats:
    try:
        reviews_df['posted'] = pd.to_datetime(reviews_df['posted'], format=date_format, errors='coerce')
        break  # Salir del bucle si la conversión es exitosa
    except ValueError:
        pass  # Continuar con el siguiente formato si hay un error

# Reemplazar el formato de fecha con el deseado
reviews_df['posted'] = reviews_df['posted'].dt.strftime('%Y-%m-%d')

# Cambiar el tipo de datos de la  columna 'item_id' a int64
reviews_df['item_id'] = reviews_df['item_id'].astype('int64')

In [ ]:
# print('games')
# print(df_games.info())
# print('items')
# print(df_items.info())
# print(df_items_info.info())
# print('users')
# print(user_df.info())
# print(reviews_df.info())

In [ ]:
from textblob import TextBlob

In [ ]:
def analyze_sentiment(text):

  blob = TextBlob(text)

  polarity = blob.sentiment.polarity

  if polarity > 0:
    sentiment = 2
  elif polarity < 0:
    sentiment = 0
  else:
    sentiment = 1

  return sentiment

In [ ]:
reviews_df['sentiment_analysis'] = reviews_df['review'].apply(analyze_sentiment)

In [ ]:
# Filtrar el DataFrame para mostrar solo las filas con item_id igual a 570
filtered_df = reviews_df

# Calcular la cantidad de elementos únicos y contar cuántos hay de cada uno
unique_sentiments_count = filtered_df['sentiment_analysis'].nunique()
sentiments_counts = filtered_df['sentiment_analysis'].value_counts()

# Calcular la mediana de la columna 'sentiment_analysis'
median_sentiment = filtered_df['sentiment_analysis'].median()

# Calcular la moda de la columna 'sentiment_analysis'
mode_sentiment = filtered_df['sentiment_analysis'].mode().values[0]

# Mostrar los resultados
print("Cantidad de elementos únicos en 'sentiment_analysis':", unique_sentiments_count)
print("Conteo de cada valor en 'sentiment_analysis':")
print(sentiments_counts)
print("Mediana de 'sentiment_analysis':", median_sentiment)
print("Moda de 'sentiment_analysis':", mode_sentiment)

Cantidad de elementos únicos en 'sentiment_analysis': 3
Conteo de cada valor en 'sentiment_analysis':
2    33531
1    13028
0    12746
Name: sentiment_analysis, dtype: int64
Mediana de 'sentiment_analysis': 2.0
Moda de 'sentiment_analysis': 2


In [ ]:
reviews_df_clean = reviews_df[['user_id', 'item_id', 'recommend', 'posted', 'sentiment_analysis']]

In [ ]:
reviews_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             59305 non-null  object
 1   item_id             59305 non-null  int64 
 2   recommend           59305 non-null  bool  
 3   posted              49186 non-null  object
 4   sentiment_analysis  59305 non-null  int64 
dtypes: bool(1), int64(2), object(2)
memory usage: 1.9+ MB


In [ ]:
reviews_df_clean.to_csv('/content/drive/MyDrive/pf_henry/datasets/reviews.csv', index=False)